In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import accuracy_score,confusion_matrix
from scipy.stats import chi2_contingency
from scipy.stats import mannwhitneyu
import math
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix
import joblib

In [2]:
df = pd.read_excel('./data/dialysis/train_dialysis.xlsx')
train_df, test_df = train_test_split(df, test_size = 0.3, random_state = 0)

In [3]:
cat_df = train_df.iloc[:,0:39]
cat_df = pd.concat([cat_df, train_df['label']], axis = 1)

cat_p_value = []
cat_df = cat_df[cat_df['label'] >= 0]
titles = list(cat_df.columns)
for i in titles:
    table = pd.crosstab(cat_df['label'],cat_df[i])
    chi2, p, dof, expected = chi2_contingency(table)
    cat_p_value.append(p) 
d1_d39_p_value = pd.DataFrame([titles, cat_p_value]).T
d1_d39_p_value = d1_d39_p_value.rename(columns = {0:'features', 1:'p_value'})
print(d1_d39_p_value)

   features   p_value
0        d1       0.0
1        d2       0.0
2        d3       0.0
3        d4  0.003324
4        d5   0.00018
5        d6       0.0
6        d7       0.0
7        d8       0.0
8        d9       0.0
9       d10       1.0
10      d11       0.0
11      d12  0.497714
12      d13  0.041647
13      d14       0.0
14      d15       0.0
15      d16       1.0
16      d17  0.693455
17      d18  0.400225
18      d19  0.540064
19      d20       0.0
20      d21       0.0
21      d22       0.0
22      d23  0.000173
23      d24  0.046561
24      d25       1.0
25      d26  0.876397
26      d27  0.032762
27      d28       0.0
28      d29       0.0
29      d30       0.0
30      d31       0.0
31      d32       0.0
32      d33       0.0
33      d34  0.007811
34      d35    0.7026
35      d36  0.432851
36      d37       0.0
37      d38  0.271168
38      d39  0.032762
39    label       0.0


In [4]:
len(d1_d39_p_value[ d1_d39_p_value['p_value'] < 0.05])

29

In [5]:
cont_df = train_df.iloc[:,39:90]
cont_df = pd.concat([cont_df, train_df['label']], axis = 1)
cont_df = cont_df[cont_df['label'] >= 0]
titles = list(cont_df.columns)

c_p_value = []
for i in titles:
    value1=[]
    value0=[]
    my_col = cont_df[[i,'label']]
    for j in range(0, my_col.shape[0]):
        if (str(my_col.iloc[j,1]) == '1'):
            value1.append(my_col.iloc[j,0])
        elif (str(my_col.iloc[j,1]) == '-1'):
            continue
        else:
            value0.append(my_col.iloc[j,0])
    result = mannwhitneyu(value0, value1, alternative= 'two-sided') 
    c_p_value.append(result[1])
c40_c90_p_value = pd.DataFrame([titles, c_p_value]).T
c40_c90_p_value = c40_c90_p_value.rename(columns = {0:'features', 1:'p_value'})
print(c40_c90_p_value)

   features   p_value
0       c40  0.821114
1       c41  0.111405
2       c42  0.288677
3       c43  0.000193
4       c44  0.508285
5       c45  0.013147
6       c46  0.045857
7       c47  0.193075
8       c48  0.402079
9       c49  0.029993
10      c50  0.000001
11      c51       0.0
12      c52  0.523141
13      c53       0.0
14      c54  0.653911
15      c55  0.000198
16      c56       0.0
17      c57  0.055049
18      c58  0.112331
19      c59       0.0
20      c60       0.0
21      c61  0.012254
22      c62       0.0
23      c63  0.074215
24      c64       0.0
25      c65       0.0
26      c66       0.0
27      c67  0.000014
28      c68  0.746743
29      c69  0.134903
30      c70       0.0
31      c71       0.0
32      c72  0.000003
33      c73       0.0
34      c74  0.000066
35      c75  0.004319
36      c76  0.961348
37      c77  0.663361
38      c78  0.004057
39      c79  0.379648
40      c80       0.0
41      c81  0.464958
42      c82  0.016304
43      c83  0.001049
44      c8

In [6]:
len(c40_c90_p_value[ c40_c90_p_value['p_value'] < 0.05])

31

In [7]:
features = list( d1_d39_p_value[ d1_d39_p_value['p_value'] < 0.05].iloc[:-1,:]['features'] ) + list( c40_c90_p_value[ c40_c90_p_value['p_value'] < 0.05].iloc[:-1,:]['features'] )

In [8]:
with open("./hw1/dialysis-binary-features.csv", "w") as f:
    for idx, feature in enumerate(features):
        if idx < len(features)-1: f.write(feature+",")
        else: f.write(feature)

In [9]:
scaler_Std = joblib.load("./hw1/dialysis-binary-std.bin")

In [10]:
X_std = pd.DataFrame(scaler_Std.transform(df.iloc[:, :-1]), columns=df.iloc[:, :-1].columns)
X_std = X_std[features]
y = df.label

In [11]:
X_std.head()

,d1,d2,d3,d4,d5,d6,d7,d8,d9,d11,...,c73,c74,c75,c78,c80,c82,c83,c84,c89,c90
0,-0.540384,-0.355325,0.982603,-0.570234,-0.215977,0.18956,0.07188,3.396669,-0.799401,1.002883,...,0.318925,-1.016918,-0.408162,-0.843985,0.020459,0.709738,-0.496845,0.73619,0.252568,-0.154431
1,-0.540384,-0.355325,0.982603,-0.570234,-0.215977,0.18956,0.07188,3.396669,-0.799401,1.002883,...,-0.179226,-1.016918,-0.408162,-0.843985,0.020459,0.709738,-0.496845,0.73619,0.252568,-0.386456
2,-0.540384,-0.355325,0.982603,-0.570234,-0.215977,0.18956,0.07188,3.396669,-0.799401,1.002883,...,-0.179226,-1.016918,-0.408162,-0.843985,0.020459,0.709738,-0.496845,0.73619,0.252568,-0.438017
3,-0.540384,-0.355325,0.982603,-0.570234,-0.215977,0.18956,0.07188,3.396669,-0.799401,1.002883,...,-0.179226,-1.016918,-0.408162,-0.843985,0.020459,0.709738,-0.496845,0.73619,0.252568,-0.386456
4,-0.540384,-0.355325,0.982603,-0.570234,-0.215977,0.18956,0.07188,3.396669,-0.799401,1.002883,...,-0.091831,-1.016918,-0.408162,-0.843985,0.020459,0.709738,-0.496845,0.73619,0.252568,-0.386456


In [12]:
svc_model = SVC(kernel = 'rbf', C = 4, gamma = 0.125)  #填入grid search的最佳C、gamma
svc_model.fit(X_std, y)

SVC(C=4, gamma=0.125)

In [13]:
train_pred = svc_model.predict(X_std)
score_train = svc_model.decision_function(X_std)
train_acc = accuracy_score(y, train_pred)
tn, fp, fn, tp = confusion_matrix(y, train_pred).ravel()
train_specificity = tn / (tn+fp)
train_sensitivity = tp / (tp+fn)
train_PPV = tp / (tp+fp)
train_NPV = tn / (fn+tn)
train_MCC = ((tp*tn)-(fp*fn))/math.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))

In [17]:
display(train_acc, tn, fp, fn, tp)

0.9244375265317674

4947

231

303

1586

In [14]:
joblib.dump(svc_model, "./hw1/dialysis-binary-svc.model")

['./hw1/dialysis-binary-svc.model']

In [15]:
knn_model = KNeighborsClassifier()
knn_model.fit(X_std,y)

KNeighborsClassifier()

In [19]:
train_pred = knn_model.predict(X_std)
# score_train = knn_model.decision_function(X_std)
train_acc = accuracy_score(y, train_pred)
tn, fp, fn, tp = confusion_matrix(y, train_pred).ravel()
train_specificity = tn / (tn+fp)
train_sensitivity = tp / (tp+fn)
train_PPV = tp / (tp+fp)
train_NPV = tn / (fn+tn)
train_MCC = ((tp*tn)-(fp*fn))/math.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))

In [20]:
display(train_acc, tn, fp, fn, tp)

0.8715154945521437

4738

440

468

1421

In [16]:
joblib.dump(svc_model, "./hw1/dialysis-binary-knn.model")

['./hw1/dialysis-binary-knn.model']